Import Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
print("Libraries Imported Successfully")

Libraries Imported Successfully


Define Data Paths

In [2]:
BASE_PATH = './Dataset/training'

IMAGE_DIR = os.path.join(BASE_PATH, 'image_2')
LIDAR_DIR = os.path.join(BASE_PATH, 'velodyne')
CALIB_DIR = os.path.join(BASE_PATH, 'calib')
LABEL_DIR = os.path.join(BASE_PATH, 'label_2')

if not os.path.exists(BASE_PATH):
    print(f"ERROR: Base path not found at '{BASE_PATH}'")
else:
    print("Data paths defined successfully")

Data paths defined successfully


Function for Data Loading

In [3]:
def read_calib_file(filepath):
    """Reads a KITTI calibration file and returns a dictionary of matrices."""
    calib = {}
    with open(filepath, 'r') as f:
        for line in f.readlines():
            if ':' in line:
                key, value = line.split(':', 1)
                calib[key] = np.array([float(x) for x in value.strip().split()])
    
    # Reshape matrices to their proper dimensions
    calib['P2'] = calib['P2'].reshape(3, 4)
    # Transformation from velodyne to camera coordinate system
    Tr_velo_to_cam = calib['Tr_velo_to_cam'].reshape(3, 4)
    # Add a row to make it a 4x4 matrix
    calib['Tr_velo_to_cam'] = np.vstack([Tr_velo_to_cam, [0, 0, 0, 1]])

    return calib

def read_label_file(filepath):
    """Reads a KITTI label file and returns a list of objects."""
    objects = []
    with open(filepath, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split(' ')
            obj = {
                'type': parts[0],
                'truncated': float(parts[1]),
                'occluded': int(parts[2]),
                'alpha': float(parts[3]),
                'bbox': np.array([float(p) for p in parts[4:8]]),
                'dimensions': np.array([float(p) for p in parts[8:11]]), # height, width, length
                'location': np.array([float(p) for p in parts[11:14]]), # x, y, z in camera coords
                'rotation_y': float(parts[14])
            }
            objects.append(obj)
    return objects

def read_lidar_file(filepath):
    """Reads a KITTI LiDAR file (.bin) and returns an (N, 4) numpy array."""
    # Each point is (x, y, z, intensity)
    return np.fromfile(filepath, dtype=np.float32).reshape(-1, 4)

print("Data loading functions are defined.")

Data loading functions are defined.


Function for Geometry and Projection

In [4]:
def create_3d_bbox_corners(obj):
    h, w, l = obj['dimensions']
    loc = obj['location']
    ry = obj['rotation_y']

    # 3D bounding box corners
    x_corners = [l/2, l/2, -l/2, -l/2, l/2, l/2, -l/2, -l/2]
    y_corners = [0, 0, 0, 0, -h, -h, -h, -h]
    z_corners = [w/2, -w/2, -w/2, w/2, w/2, -w/2, -w/2, w/2]

    corners_3d = np.vstack([x_corners, y_corners, z_corners])

    # Rotation matrix around the Y-axis
    R = np.array([[np.cos(ry), 0, np.sin(ry)],
                  [0, 1, 0],
                  [-np.sin(ry), 0, np.cos(ry)]])

    # Rotate and translate the corners
    corners_3d = R @ corners_3d

    # [rotated_box_x_corners; rotated_box_y_corners; rotated_box_z_corners] = [rotated_box_x_corners+x_centroid_camera_coord_frame, rotated_box_y_corners+y_centroid_camera_coord_frame, rotated_box_z_corners+z_centroid_camera_coord_frame]
    corners_3d += loc.reshape(3, 1)
    
    return corners_3d.T

def project_cam_to_velo(points_cam, calib):
    # Transform 3D points from camera coordinates to velodyne coordinates
    T_velo_to_cam = calib['Tr_velo_to_cam']
    
    T_cam_to_velo = np.linalg.inv(T_velo_to_cam)

    # Convert points to homogeneous coordinates
    points_h = np.hstack([points_cam, np.ones((points_cam.shape[0], 1))])
    # Apply transformation
    points_velo = (T_cam_to_velo @ points_h.T).T
    return points_velo[:, :3]


def project_3d_to_2d(points_3d, calib):
    """Projects 3D points in camera coordinates to 2D image coordinates."""
    # Convert to homogeneous coordinates
    points_h = np.hstack([points_3d, np.ones((points_3d.shape[0], 1))])
    
    # Project using P2 matrix (3x4 camera projection matrix)
    points_2d_h = calib['P2'] @ points_h.T  # 3xN
    
    # Convert to non-homogeneous coordinates
    points_2d = points_2d_h.T
    points_2d[:, 0] /= points_2d[:, 2]  # x = x/z
    points_2d[:, 1] /= points_2d[:, 2]  # y = y/z
    
    return points_2d[:, :2]  # Return only x, y coordinates


print("Geometry and projection functions are defined.")

Geometry and projection functions are defined.


Bounding Box Drawing Function

In [5]:
color_map = {
        'Car': 'red',
        'Pedestrian': 'green',
        'Cyclist': 'blue',
        'Van': 'orange',
        'Truck': 'purple',
    }

def draw_2d_bbox(ax, obj):
    """Draws a 2D bounding box on a matplotlib axes."""
    from matplotlib.patches import Rectangle

    # Get bbox coordinates
    xmin, ymin, xmax, ymax = obj['bbox']
    width = xmax - xmin
    height = ymax - ymin

    # Draw rectangle
    rect = Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='lime', facecolor='none')
    ax.add_patch(rect)

    global color_map
    color = color_map.get(obj['type'], 'cyan')
    # Add object type label
    ax.text(xmin, ymin - 10, f"{obj['type']}", color=color, fontsize=12,
            bbox=dict(facecolor='black', alpha=0.5, lw=0))


def draw_3d_bbox(ax, obj, calib, image_width, image_height):
    """Draws a 3D bounding box on the camera image."""
     # Create 3D bounding box corners in camera coordinates
    corners_3d_cam = create_3d_bbox_corners(obj)
    
    # Project to 2D image coordinates
    corners_2d = project_3d_to_2d(corners_3d_cam, calib)
    
    # Define the edges of the 3D box
    edges = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
        [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
        [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical edges
    ]
    
    # Color map for different object types
    global color_map
    
    color = color_map.get(obj['type'], 'cyan')
    
    # Draw all edges with visibility check
    for edge in edges:
        start_idx, end_idx = edge
        
        x1, y1 = corners_2d[start_idx, 0], corners_2d[start_idx, 1]
        x2, y2 = corners_2d[end_idx, 0], corners_2d[end_idx, 1]
        
        # Check if either endpoint is within the image
        start_visible = (0 <= x1 <= image_width and 0 <= y1 <= image_height)
        end_visible = (0 <= x2 <= image_width and 0 <= y2 <= image_height)
        
        # If both points are outside the image, skip this edge
        if not start_visible and not end_visible:
            continue
            
        # Clip coordinates to image boundaries
        x1_clipped = np.clip(x1, 0, image_width - 1)
        y1_clipped = np.clip(y1, 0, image_height - 1)
        x2_clipped = np.clip(x2, 0, image_width - 1)
        y2_clipped = np.clip(y2, 0, image_height - 1)
        
        # Draw the line
        ax.plot([x1_clipped, x2_clipped], [y1_clipped, y2_clipped], 
               color=color, linewidth=2, alpha=0.8)


print("Drawing functions are defined.")

Drawing functions are defined.


Main Visualization Function


In [ ]:

def visualize_data(sample_id):
    """Main visualization function with properly aligned BEV and flipped x-axis"""

    # Construct file paths
    image_file = os.path.join(IMAGE_DIR, f"{sample_id}.png")
    lidar_file = os.path.join(LIDAR_DIR, f"{sample_id}.bin")
    calib_file = os.path.join(CALIB_DIR, f"{sample_id}.txt")
    label_file = os.path.join(LABEL_DIR, f"{sample_id}.txt")

    # Check if all files exist
    for f in [image_file, lidar_file, calib_file, label_file]:
        if not os.path.exists(f):
            print(f"Error: File not found at '{f}'")
            return

    # Load data
    image = Image.open(image_file)
    points = read_lidar_file(lidar_file)
    calib = read_calib_file(calib_file)
    objects = read_label_file(label_file)

    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    fig.suptitle(f'KITTI Sample: {sample_id}', fontsize=16)

    # Camera Image with Bounding Boxes
    ax1.imshow(image)
    ax1.set_title('Camera Image with Bounding Boxes', fontsize=12)
    ax1.axis('off')


    # Define BEV range
    x_range = (0, 80)      # Forward distance (0-80 meters)
    y_range = (-60, 60)    # Side distance (-60 to 60 meters)
    
    # Filter LiDAR points within BEV range
    mask = ((points[:, 0] >= x_range[0]) & (points[:, 0] <= x_range[1]) &
            (points[:, 1] >= y_range[0]) & (points[:, 1] <= y_range[1]))
    filtered_points = points[mask]

    # Plot LiDAR points
    scatter = ax2.scatter(filtered_points[:, 1],  # y (left/right) -> x-axis
                         filtered_points[:, 0],   # x (forward) -> y-axis
                         c=filtered_points[:, 2], # z (height) for color
                         cmap='viridis', s=1, alpha=0.7)

    # Add colorbar for height
    cbar = plt.colorbar(scatter, ax=ax2)
    cbar.set_label('Height (meters)', rotation=270, labelpad=15)

    # Set axis limits
    ax2.set_xlim(y_range[1], y_range[0])
    ax2.set_ylim(x_range[0], x_range[1])
    ax2.set_xlabel('Y (meters) - Left/Right')
    ax2.set_ylabel('X (meters) - Forward')
    ax2.set_title('LiDAR Bird\'s-Eye View with 3D Bounding Boxes', fontsize=12)
    ax2.grid(True, alpha=0.3)

    # --- Draw 3D Bounding Boxes on BEV ---
    global color_map
    image_width, image_height = image.size
    for obj in objects:
        obj_type = obj['type']
        if obj_type in ['Car', 'Van', 'Truck', 'Pedestrian', 'Cyclist']:
            try:
                draw_2d_bbox(ax1, obj)

                draw_3d_bbox(ax1, obj, calib, image_width, image_height)
                # Create 3D bounding box corners in camera coordinates
                corners_3d_cam = create_3d_bbox_corners(obj)



                # Project to velodyne coordinates
                corners_3d_velo = project_cam_to_velo(corners_3d_cam, calib)

                # Get color for this object type
                color = color_map.get(obj_type, 'cyan')

                # Use bottom corners for BEV (first 4 points)
                bottom_corners = corners_3d_velo[:4, :2]  # x, y coordinates

                # Extract coordinates
                x_coords = bottom_corners[:, 0]  # forward distance
                y_coords = bottom_corners[:, 1]  # left/right distance

                # Draw bounding box as polygon
                from matplotlib.patches import Polygon
                poly = Polygon(np.column_stack([y_coords, x_coords]),
                              closed=True, edgecolor=color, facecolor='none',
                              linewidth=2, alpha=0.8, label=obj_type)
                ax2.add_patch(poly)

                # Add object type label at center
                center_y = np.mean(y_coords)
                center_x = np.mean(x_coords)

                # Only add label if within view
                if (center_x >= x_range[0] and center_x <= x_range[1] and
                    center_y >= y_range[0] and center_y <= y_range[1]):
                    ax2.text(center_y, center_x, obj_type, color=color, fontsize=9,
                            bbox=dict(facecolor='black', alpha=0.7, boxstyle='round,pad=0.2'),
                            ha='center', va='center')

            except Exception as e:
                print(f"Error drawing {obj_type}: {e}")
                continue

    # Add legend for object types
    unique_types = list(set([obj['type'] for obj in objects if obj['type'] in color_map]))
    if unique_types:
        legend_elements = [plt.Line2D([0], [0], color=color_map[obj_type], lw=2, label=obj_type)
                          for obj_type in unique_types if obj_type in color_map]
        ax2.legend(handles=legend_elements, loc='upper right', framealpha=0.9)

    plt.tight_layout()
    plt.show()


print("Main visualization function is defined.")

Main visualization function is defined.


In [ ]:
try:
    sample_ids = sorted([f.split('.')[0] for f in os.listdir(IMAGE_DIR) if f.endswith('.png')])
    if not sample_ids:
        print("No samples found. Please check your IMAGE_DIR path and data.")
    else:
        # Interactive Widget
        sample_dropdown = widgets.Dropdown(
            options=sample_ids,
            value=sample_ids[0], # Default to the first sample
            description='Sample ID:',
            disabled=False,
            style={'description_width': 'initial'}
        )

        interactive_plot = widgets.interactive(visualize_data, sample_id=sample_dropdown)
        display(interactive_plot)

except FileNotFoundError:
    print(f"ERROR: Could not find the image directory at '{IMAGE_DIR}'.")
    print("Please ensure the path is correct in Cell 2 and that the directory exists.")

interactive(children=(Dropdown(description='Sample ID:', options=('000000', '000001', '000002', '000003', '000…